# Estimación de costos: Proyecto de Análisis Genómico: Biotech Explorer Corp

Estimación AWS: https://calculator.aws/#/estimate?id=ef7f84aa8f064d58cef41f09684759c72ebb44aa

Análisis y descripción de los costos de la arquitectura planteada para el caso **Biotech Explorer Corp**.

La arquitectura se divide en tres partes principales:

1. Acceso a los datos (API de terceros y almacenamiento de datos genómicos crudos en S3)
2. Preprocesamiento y entrenamiento (procesamiento intensivo de datos, almacenamiento compartido, y uso de servicios de bases de datos)
3. Acceso a resultados y predicciones (entrega de resultados procesados y modelos entrenados para hacer predicciones)


In [32]:
import pandas as pd

In [33]:
config = []

## 1. Acceso a los Datos

### Amazon EC2 (t4g.micro)

In [34]:
ec2_get_data_instance = {
    "Service Name": "Amazon EC2 ",
    "Group": "Data Access",
    "Description": "EC2 Biotech Get Data",
    "Region": "US East (N. Virginia)",
    "Service Cost": {
        "monthly": "9.15",
        "upfront": "0.00",
        "12 months": "109.80"
    },
    "Properties": {
        "Tenancy": "Shared Instances",
        "Operating system": "Linux",
        "Workload": "Consistent, Number of instances: 1",
        "Advance EC2 instance": "t4g.micro",
        "Pricing strategy": "Standard Reserved Instances 3yr No Upfront",
        "Enable monitoring": "disabled",
        "EBS Storage amount": "50 GB",
        "DT Inbound: All other regions": "500 TB per month",
        "DT Outbound: Amazon CloudFront": "50 GB per month",
        "DT Intra-Region:": "0 TB per month"
    }
}

config.append(ec2_get_data_instance)

Para la tarea de recoger datos de la API de terceros, hemos seleccionado una instancia t4g.micro de EC2. Esta instancia es suficiente ya que solo requiere acceder a la API periódicamente para obtener los datos genómicos y enviarlos a Amazon S3.

### Amazon S3 Standard

In [35]:
s3_standard = {
    "Service Name": "S3 Standard",
    "Group": "Data Access",
    "Description": "S3 Biotech",
    "Region": "US East (N. Virginia)",
    "Service Cost": {
        "monthly": "24.27",
        "upfront": "0.00",
        "12 months": "291.24"
    },
    "Properties": {
        "S3 Standard storage": "1 TB per month",
        "PUT, COPY, POST, LIST requests to S3 Standard": "1000",
        "GET, SELECT, and all other requests from S3 Standard": "1000",
        "Data returned by S3 Select": "1 TB per month"
    }
}
config.append(s3_standard)

Los datos genómicos recogidos de la API se almacenan en Amazon S3 en su formato crudo. Seleccionamos S3 Standard porque los datos en esta etapa son accedidos frecuentemente durante el preprocesamiento y el entrenamiento de modelos.

### S3 Glacier Flexible Retrieval

In [36]:
s3_glacier = {
    "Service Name": "S3 Glacier Flexible Retrieval",
    "Group": "Data Access",
    "Description": "S3 Biotech",
    "Region": "US East (N. Virginia)",
    "Service Cost": {
        "monthly": "6.91",
        "upfront": "0.00",
        "12 months": "82.92"
    },
    "Properties": {
        "S3 Glacier Flexible Retrieval Average Object Size": "16 MB",
        "Provisioned Capacity Units": "0",
        "S3 Glacier Flexible Retrieval storage": "500 GB per month",
        "PUT, COPY, POST, LIST requests to S3 Glacier Flexible Retrieval": "10",
        "Lifecycle Transitions into S3 Glacier Flexible Retrieval": "10",
        "Restore requests (Standard)": "10",
        "Restore requests (Expedited)": "10",
        "Restore requests (Bulk)": "10",
        "Data retrievals (Standard)": "500 GB per month",
        "Data retrievals (Bulk)": "500 GB per month"
    }
}
config.append(s3_glacier)

Hemos seleccionado el uso de S3 Glacier Flexible Retrieval para almacenar datos genómicos antiguos que no se acceden con frecuencia. Para almacenar los datos a un costo significativamente más bajo, con tiempos de respuesta flexibles.

## 2. Preprocesamiento y Entrenamiento

### Amazon EC2 (t4g.xlarge)

In [37]:
ec2_preprocess_data_instance = {
    "Service Name": "Amazon EC2 ",
    "Group": "Data Processing and Analysis",
    "Description": "EC2 Biotech Preprocess Data",
    "Region": "US East (N. Virginia)",
    "Service Cost": {
        "monthly": "48.86",
        "upfront": "0.00",
        "12 months": "586.32"
    },
    "Properties": {
        "Tenancy": "Shared Instances",
        "Operating system": "Linux",
        "Workload": "Consistent, Number of instances: 1",
        "Advance EC2 instance": "t4g.xlarge",
        "Pricing strategy": "Standard Reserved Instances 3yr No Upfront",
        "Enable monitoring": "disabled",
        "EBS Storage amount": "50 GB",
        "DT Inbound: All other regions": "500 TB per month",
        "DT Outbound: Amazon CloudFront": "50 GB per month",
        "DT Intra-Region:": "0 TB per month"
    }
}
config.append(ec2_preprocess_data_instance)

Para el preprocesamiento de los datos genómicos antes del entrenamiento, hemos elegido una instancia t4g.xlarge, una instancia de tamaño mediano basada en ARM que ofrece una buena relación de costo-beneficio. Hemos cotizado para tres años, lo que reduce costos significativamente con respecto a las instancias bajo demanda.


### Amazon Elastic File System (EFS)

In [38]:
efs = {
    "Service Name": "Amazon Elastic File System (EFS)",
    "Group": "Data Processing and Analysis",
    "Region": "US East (N. Virginia)",
    "Service Cost": {
        "monthly": "17.40",
        "upfront": "0.00",
        "12 months": "208.80"
    },
    "Properties": {
        "Desired Storage Capacity": "200 GB per month",
        "Read Data Transferred": "100 GB per month",
        "Write Data Transferred": "200 GB per month"
    }
}
config.append(efs)

Hemos elegido EFS para almacenamiento compartido entre las instancias EC2 que participan en el preprocesamiento y entrenamiento de los modelos. Esto permite que todos los nodos de procesamiento accedan a los mismos archivos temporales y de trabajo durante el proceso de análisis.

### Amazon RDS (PostgreSQL)

In [39]:
rds = {
    "Service Name": "Amazon RDS for PostgreSQL",
    "Group": "Data Processing and Analysis",
    "Description": "RDS for PostgreSQL Biotech",
    "Region": "US East (N. Virginia)",
    "Service Cost": {
        "monthly": "307.50",
        "upfront": "0.00",
        "12 months": "3690.00"
    },
    "Properties": {
        "Storage amount": "100 GB",
        "Storage volume": "General Purpose SSD (gp2)",
        "Nodes": "2",
        "Instance Type": "db.t3.medium",
        "Utilization (On-Demand only)": "100 %Utilized/Month",
        "Deployment Option": "Multi-AZ",
        "Pricing Model": "OnDemand"
    }
}
config.append(rds)

Hemos seleccionado Amazon RDS con PostgreSQL para almacenar los resultados estructurados del entrenamiento de los modelos. Configuramos Multi-AZ para garantizar la alta disponibilidad, ya que no estamos dispuestos a perder datos de entrenamiento y resultados.

### AWS Batch

AWS Batch es el servicio clave para la gestión de cargas de trabajo por lotes (no es un servicio configurable en AWS calculator), ya que permite automatizar el escalado de las instancias EC2 en función de la cantidad de datos que se deben procesar. AWS Batch decide cuántas instancias EC2 se necesitan y las escala automáticamente según la carga de trabajo.

**Importante**:

AWS Batch puede aumentar los costos debido al uso de más instancias EC2 cuando las cargas de datos genómicos son mayores.

## 3. Acceso a Resultados y Predicciones

### Amazon EKS

In [40]:
eks = {
    "Service Name": "Amazon EKS",
    "Group": "Model Prediction",
    "Description": "Kubernetes Biotech",
    "Region": "US East (N. Virginia)",
    "Service Cost": {
        "monthly": "438.00",
        "upfront": "0.00",
        "12 months": "5256.00"
    },
    "Properties": {
        "Number of EKS Clusters": "1"
    }
}
config.append(eks)

Seleccionamos EKS para orquestar los contenedores Docker que realizan las predicciones basadas en los modelos entrenados. EKS maneja de forma automática la distribución y el escalado de las predicciones en contenedores.

### Amazon EC2 (t4g.xlarge)

In [41]:
ec2_deploy_model_instance = {
    "Service Name": "Amazon EC2 ",
    "Group": "Model Prediction",
    "Description": "EC2 Biotech despliegue",
    "Region": "US East (N. Virginia)",
    "Service Cost": {
        "monthly": "48.86",
        "upfront": "0.00",
        "12 months": "586.32"
    },
    "Properties": {
        "Tenancy": "Shared Instances",
        "Operating system": "Linux",
        "Workload": "Consistent, Number of instances: 1",
        "Advance EC2 instance": "t4g.xlarge",
        "Pricing strategy": "Standard Reserved Instances 3yr No Upfront",
        "Enable monitoring": "disabled",
        "EBS Storage amount": "50 GB",
        "DT Inbound: All other regions": "500 TB per month",
        "DT Outbound: Amazon CloudFront": "50 GB per month",
        "DT Intra-Region:": "0 TB per month"
    }
}
config.append(ec2_deploy_model_instance)

Hemos seleccionado una instancia de EC2 t4g.xlarge para la ejecución de modelos de predicción. Esta instancia ofrece un buen rendimiento para generar predicciones basadas en los datos que han sido preprocesados y los modelos entrenados.

## General

In [42]:
cloud_watch = {
    "Service Name": "Amazon CloudWatch",
    "Group": "General",
    "Description": "Monitoreo Biotech ",
    "Region": "US East (N. Virginia)",
    "Service Cost": {
        "monthly": "147.85",
        "upfront": "0.00",
        "12 months": "1774.20"
    },
    "Properties": {
        "Number of Metrics (includes detailed and custom metrics)": "20",
        "GetMetricData: Number of metrics requested": "100",
        "Number of other API requests": "100",
        "Standard Logs: Data Ingested": "100 GB",
        "Logs Delivered to CloudWatch Logs: Data Ingested": "100 GB",
        "Logs Delivered to S3: Data Ingested": "100 GB",
        "Expected Logs Data scanned": "50 GB",
        "Number of Dashboards": "2",
        "Number of Standard Resolution Alarm Metrics": "5",
        "Number of High Resolution Alarm Metrics": "2",
        "Number of monitored resources": "2 per hour"
    }
}
config.append(cloud_watch)

Hemos elegido CloudWatch para monitorear todo el sistema, generar métricas, y enviar alertas si algún servicio o recurso tiene problemas de rendimiento o disponibilidad.

## Visualización de costos

In [47]:
df = pd.DataFrame(config)

# Convertir las columnas de costos a tipo numérico
df['monthly'] = pd.to_numeric(df['Service Cost'].apply(lambda x: x['monthly']))
df['12 months'] = pd.to_numeric(df['Service Cost'].apply(lambda x: x['12 months'])) 

df['Instance Type'] = df['Properties'].apply(lambda x: x.get('Instance Type', x.get('Advance EC2 instance', '')))
df['Storage amount'] = df['Properties'].apply(lambda x: x.get('Storage amount', x.get('Desired Storage Capacity', x.get('S3 Standard storage', x.get('S3 Glacier Flexible Retrieval storage', x.get('EBS Storage amount', ''))))))

# Remover columna Service Cost
df.drop(columns=['Service Cost'], inplace=True)

# Remover columna Properties
df.drop(columns=['Properties'], inplace=True)
df


,Service Name,Group,Description,Region,monthly,12 months,Instance Type,Storage amount
0,Amazon EC2,Data Access,EC2 Biotech Get Data,US East (N. Virginia),9.15,109.80,t4g.micro,50 GB
1,S3 Standard,Data Access,S3 Biotech,US East (N. Virginia),24.27,291.24,,1 TB per month
2,S3 Glacier Flexible Retrieval,Data Access,S3 Biotech,US East (N. Virginia),6.91,82.92,,500 GB per month
3,Amazon EC2,Data Processing and Analysis,EC2 Biotech Preprocess Data,US East (N. Virginia),48.86,586.32,t4g.xlarge,50 GB
4,Amazon Elastic File System (EFS),Data Processing and Analysis,NaN,US East (N. Virginia),17.40,208.80,,200 GB per month
5,Amazon RDS for PostgreSQL,Data Processing and Analysis,RDS for PostgreSQL Biotech,US East (N. Virginia),307.50,3690.00,db.t3.medium,100 GB
6,Amazon EKS,Model Prediction,Kubernetes Biotech,US East (N. Virginia),438.00,5256.00,,
7,Amazon EC2,Model Prediction,EC2 Biotech despliegue,US East (N. Virginia),48.86,586.32,t4g.xlarge,50 GB
8,Amazon CloudWatch,General,Monitoreo Biotech,US East (N. Virginia),147.85,1774.20,,


### Costos totales

In [51]:
total_monthly = df['monthly'].sum()
total_12_months = df['12 months'].sum()

total_monthly, total_12_months

(np.float64(1048.8), np.float64(12585.6))

Costo total mensual: $1,048.8  
Costo total anual: $12,585.6

### Costos por grupo

In [49]:
# Agrupar por "Group" y calcular el total por grupo
grouped_df = df.groupby('Group').sum()

# Mostrar los totales por grupo
grouped_df.reset_index(inplace=True)
grouped_df

,Group,Service Name,Description,Region,monthly,12 months,Instance Type,Storage amount
0,Data Access,Amazon EC2 S3 StandardS3 Glacier Flexible Retr...,EC2 Biotech Get DataS3 BiotechS3 Biotech,US East (N. Virginia)US East (N. Virginia)US E...,40.33,483.96,t4g.micro,50 GB1 TB per month500 GB per month
1,Data Processing and Analysis,Amazon EC2 Amazon Elastic File System (EFS)Ama...,EC2 Biotech Preprocess DataRDS for PostgreSQL ...,US East (N. Virginia)US East (N. Virginia)US E...,373.76,4485.12,t4g.xlargedb.t3.medium,50 GB200 GB per month100 GB
2,General,Amazon CloudWatch,Monitoreo Biotech,US East (N. Virginia),147.85,1774.20,,
3,Model Prediction,Amazon EKSAmazon EC2,Kubernetes BiotechEC2 Biotech despliegue,US East (N. Virginia)US East (N. Virginia),486.86,5842.32,t4g.xlarge,50 GB


Con base en los ingresos y las ganancias de la empresa, esta estimación es muy razonable y sostenible. Los costos representan una pequeña fracción de las ganancias anuales ($5'000,0000), lo que garantiza que la infraestructura no se convierta en una carga financiera. 